In [34]:
import ee

In [35]:
ee.Authenticate()

True

In [36]:
ee.Initialize()

In [38]:
aoi = ee.Geometry.Polygon([
    [
        [-71.07505705182442,42.226543564466205],
        [-70.99386122052559,42.226543564466205],
        [-70.99386122052559,42.26682587484719],
        [-71.07505705182442,42.26682587484719],
        [-71.07505705182442,42.226543564466205]
    ]
])




In [39]:
from IPython.display import Image

In [40]:
# Select a Sentinel-2 image
sentinel2 = ee.Image(ee.ImageCollection('COPERNICUS/S2')
                     .filterBounds(aoi)
                     .filterDate('2023-01-01', '2023-01-30')
                     .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5))
                     .first())

# Visualization parameters for Sentinel-2
vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2'],  # RGB
}

# Overlay the AOI on the Sentinel-2 image
aoi_layer = ee.Image().paint(featureCollection=aoi, color=1, width=3)
composite = sentinel2.visualize(**vis_params).blend(aoi_layer.visualize(**{'palette': 'FF0000'}))

# Get the URL for the thumbnail of the composite image
thumbnail_url = composite.getThumbURL({
    'region': aoi.bounds().getInfo()['coordinates'],
    'dimensions': 512
})

# Display the image directly in the notebook
display(Image(url=thumbnail_url))

In [41]:
import folium


In [42]:
aoi_coordinates =[
    
    
 [-71.07505705182442,42.226543564466205],
 [-70.99386122052559,42.226543564466205],
 [-70.99386122052559,42.26682587484719],
 [-71.07505705182442,42.26682587484719],
 [-71.07505705182442,42.226543564466205]   # Closing the loop
    
    
]
# Calculate the center of your AOI for the map's initial view
lat_center = sum(lat for lat, lon in aoi_coordinates) / len(aoi_coordinates)
lon_center = sum(lon for lat, lon in aoi_coordinates) / len(aoi_coordinates)

# Create a Folium map centered around the middle of the AOI
m = folium.Map(location=[lat_center, lon_center], zoom_start=15)

# Add a polygon overlay to the map using the AOI coordinates
folium.Polygon(
    locations=aoi_coordinates,  # Use the AOI coordinates directly
    color='blue',
    weight=3,
    fill_color='blue',
    fill_opacity=0.1,
).add_to(m)

# Display the map
m

In [32]:
# Filter the Sentinel-2 ImageCollection for images within the AOI and time frame, and compute the median
sentinelCollection = ee.ImageCollection("COPERNICUS/S2_SR") \
    .filterDate('2024-01-01', '2024-01-30') \
    .filterBounds(aoi) \
    .median()

# Clip the median image to the AOI
clippedImage = sentinelCollection.clip(aoi)

# Function to cast all bands to Float32
def castImage(image):
    return image.toFloat()

# Cast all bands of the clipped image to Float32
castedImage = castImage(clippedImage)


 #Export the casted image, specifying scale and region (this is an example, see below for details)
task = ee.batch.Export.image.toDrive(**{
   'image': castedImage,
   'description': 'boston-2-Full-Bands',
   'scale': 10,
   'region': aoi.getInfo()['coordinates'],
   'fileFormat': 'GeoTIFF',
   'maxPixels': 1e9
 })
task.start()